## <a> Segmentação de Clientes </a>

No universo dinâmico dos negócios, compreender as nuances individuais dos clientes é crucial para o sucesso empresarial. Após uma eficiente análise de componentes principais (PCA) que reduziu nosso conjunto de dados de 30 perguntas para 5 componentes principais, avançamos para a segunda etapa: a segmentação de clientes.

Utilizaremos técnicas de clusterização, com destaque para o algoritmo K-means, para agrupar clientes com caracaterísticas similares. Dessa forma, uma campanha de marketing pode ser melhor direcionada e atingir os segmentos de consumidores que possuem maior chance de adqurir nosso produto. Essa abordagem não só simplifica estratégias de marketing e atendimento, como também maximiza a eficiência operacional, sobretudo quando o nível de recursos destinados à publicidade não é elevado.

Ao criar grupos homogêneos, antecipamos necessidades específicas e ajustamos nossas abordagens de forma mais precisa. Estamos prestes a explorar padrões subjacentes, permitindo uma personalização mais profunda em nossas interações com os clientes. Este é o caminho para estratégias mais inteligentes, focadas em construir relacionamentos duradouros e satisfazer as demandas individuais de nossos clientes. Vamos agora utilizar o potencial do K-means para impulsionar o sucesso da microvan, mantendo o cliente no epicentro de nossas decisões estratégicas.

É importante lembrar as diferenças entre redução de dimensionalidade e clusterização:

> A redução de dimensionalidade é utilizada para diminuir o número de features em um DataFrame. A utilização do PCA retorna componentes principais a partir de um número maior de colunas.

> A clusterização, por sua vez, agrupa as linhas do DataFrame (isto é, as observações) de acordo com suas características (valores das features).

> Na primeira parte do projeto (no notebook PCA - Redução Dimensionalidade) foi realizada a redução de dimensionalidade para que se tornasse mais fácil interpretar os resultados provenientes da clusterização. Ao trocar 30 colunas por 5 componentes principais, é possível trazer mais clareza para a análise de segmentação.

In [13]:
# importando bibliotecas necessárias
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Setando configurações do pandas para mostrar todas as linhas e colunas 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Setando configurações no pandas para não mostrar números com notação científica
pd.set_option('display.float_format', lambda x: "%.2f" % x) # Mostrando numeros com apenas 2 casas decimais

In [4]:
# Acessando arquivos
with open("principal_components.pkl", 'rb') as pca_file:
    principal_components = pickle.load(pca_file)

In [5]:
# Este é o dataframe do questionário de 30 perguntas transformado em 5 componentes principais
principal_components

# Obs: Cada linha desse array corresponde à resposta de determinado indivíduo às metaperguntas geradas a partir do PCA

array([[ 4.95739724, -0.630778  ,  0.25397408,  4.53413096,  0.4787577 ],
       [-0.42529226, -4.57730748, -2.68257259, -2.62292284, -0.50422577],
       [-4.20223653,  5.99835902,  2.46455968,  1.54782301, -2.41202492],
       ...,
       [-6.4115679 , -4.47822563, -6.6555251 , -0.29161379, -1.62472682],
       [-3.57270499, -5.5292335 ,  0.25267482,  2.61167859, -1.04362801],
       [ 4.05034892,  2.45977589,  3.75878157, -0.49039215,  0.74268315]])

#### Ex: a última linha corresponde a um cliente que respondeu o questionário e tem o componente principal 1 no valor de 4.05, ou seja, um valor forte positivo para "Workaholics Econômicos" e valor 3.75 para "Família ativa" 

In [19]:
# Carregando base de dados novamente
df = pd.read_csv('./data/microvan.csv')

In [20]:
# Setando coluna subjnumb (subject number) como índice da base de dados
df = df.set_index("subjnumb")

In [21]:
# De posse do array correspondente às respostas (nos componentes principais), vamos transformá-lo num DataFrame para nomear as
# colunas
# Definindo nomes dos componentes
nomes_componentes = ['Workaholics', 'Carros_Pequenos', 'Família', 'Velozes', 'Carro_Caro']

# Transformando num pandas DataFrame
pca_df = pd.DataFrame(data = principal_components, columns = nomes_componentes, index = df.index)

# Visualizando 
pca_df.head()

,Workaholics,Carros_Pequenos,Família,Velozes,Carro_Caro
subjnumb,,,,,
1,4.96,-0.63,0.25,4.53,0.48
2,-0.43,-4.58,-2.68,-2.62,-0.50
3,-4.20,6.00,2.46,1.55,-2.41
4,4.89,-0.93,-0.76,-3.58,3.52
5,0.12,5.55,-1.02,0.07,-0.84


In [22]:
# Definindo as features demograficas novamente
features_demograficas = ['age', 'income', 'miles', 'numkids', 'female', 'educ', 'recycle', 'mvliking']

In [23]:
# juntando pca_df com colunas demográficas 
df_pca_final = pd.concat([df.loc[:, features_demograficas], pca_df], axis = 1)

# Visualizando
df_pca_final.head()

,age,income,miles,numkids,female,educ,recycle,mvliking,Workaholics,Carros_Pequenos,Família,Velozes,Carro_Caro
subjnumb,,,,,,,,,,,,,
1,34,59,14,1,0,1,3,4,4.96,-0.63,0.25,4.53,0.48
2,36,79,16,1,0,3,4,6,-0.43,-4.58,-2.68,-2.62,-0.50
3,51,72,22,2,1,3,3,3,-4.20,6.00,2.46,1.55,-2.41
4,32,42,12,0,0,3,2,3,4.89,-0.93,-0.76,-3.58,3.52
5,40,75,17,1,1,3,4,1,0.12,5.55,-1.02,0.07,-0.84


Dispondo deste DataFrame completo, com informações acerca das características demográficas de cada indivíduo, bem como suas respostas à cada metapergunta construída a partir do PCA, é possível começar a análise de clusterização com o algoritmo K-Means.

### <a> Clusterização </a>

PUSH 05